## Reference:
- https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)

        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [7]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [8]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [9]:
[print(param.size()) for param in params]

torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


[None, None, None, None, None, None, None, None, None, None]

In [10]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0681, -0.0350,  0.0099,  0.1351, -0.0954, -0.0177, -0.0361,  0.0252,
          0.0541, -0.0559]], grad_fn=<AddmmBackward>)


In [11]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [12]:
print(out)

tensor([[-0.0681, -0.0350,  0.0099,  0.1351, -0.0954, -0.0177, -0.0361,  0.0252,
          0.0541, -0.0559]], grad_fn=<AddmmBackward>)


In [14]:
print(torch.randn(1, 10))

tensor([[ 0.1282,  0.3117,  0.8717, -0.8477, -1.7905, -0.8133,  0.2588, -0.3660,
          0.7092,  0.8074]])


In [15]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5518, grad_fn=<MseLossBackward>)


In [16]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [17]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0020,  0.0028, -0.0093,  0.0014,  0.0055,  0.0035])


In [18]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [19]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

In [21]:
loss.data

tensor(0.5479)

In [22]:
output

tensor([[-0.0673, -0.0315,  0.0081,  0.1405, -0.0939, -0.0160, -0.0402,  0.0252,
          0.0525, -0.0577]], grad_fn=<AddmmBackward>)